In [1]:
import pandas as pd
import requests
import time

# === CONFIG ===
API_KEY = "PJcSVU8gR4wG9S9HVANWyssG62n2M6Sy"
TICKERS_URL = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
LIMIT = 1  # number of past reports (1 = latest)
SLEEP_TIME = 1  # avoid rate limit

# === LOAD TICKERS ===
print("Loading S&P 500 tickers...")
tickers_df = pd.read_csv(TICKERS_URL)
tickers = tickers_df['Symbol'].tolist()

# === FUNCTION TO FETCH DATA ===
def get_fmp_data(endpoint, symbol):
    url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{symbol}?limit={LIMIT}&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]
    return {}

# === MAIN LOOP ===
all_data = []

print("Fetching data for each ticker...")
for i, symbol in enumerate(tickers[70:150]):
    print(f"[{i+1}/{len(tickers)}] {symbol}")

    income = get_fmp_data("income-statement", symbol)
    balance = get_fmp_data("balance-sheet-statement", symbol)
    cashflow = get_fmp_data("cash-flow-statement", symbol)

    if income or balance or cashflow:
        merged = {
            "symbol": symbol,
            "reportDate": income.get("date", balance.get("date", cashflow.get("date", None))),
            **income,
            **{f"balance_{k}": v for k, v in balance.items()},
            **{f"cashflow_{k}": v for k, v in cashflow.items()}
        }
        all_data.append(merged)

    time.sleep(SLEEP_TIME)

# === SAVE RESULTS ===
df_70_150 = pd.DataFrame(all_data)
df_70_150.to_csv("70_150_sp500_fundamentals.csv", index=False)
print("✅ Data saved to '70_150_sp500_fundamentals.csv'")


Loading S&P 500 tickers...
Fetching data for each ticker...
[1/503] BMY
[2/503] AVGO
[3/503] BR
[4/503] BRO
[5/503] BF.B
[6/503] BLDR
[7/503] BG
[8/503] BXP
[9/503] CHRW
[10/503] CDNS
[11/503] CZR
[12/503] CPT
[13/503] CPB
[14/503] COF
[15/503] CAH
[16/503] KMX
[17/503] CCL
[18/503] CARR
[19/503] CAT
[20/503] CBOE
[21/503] CBRE
[22/503] CDW
[23/503] COR
[24/503] CNC
[25/503] CNP
[26/503] CF
[27/503] CRL
[28/503] SCHW
[29/503] CHTR
[30/503] CVX
[31/503] CMG
[32/503] CB
[33/503] CHD
[34/503] CI
[35/503] CINF
[36/503] CTAS
[37/503] CSCO
[38/503] C
[39/503] CFG
[40/503] CLX
[41/503] CME
[42/503] CMS
[43/503] KO
[44/503] CTSH
[45/503] CL
[46/503] CMCSA
[47/503] CAG
[48/503] COP
[49/503] ED
[50/503] STZ
[51/503] CEG
[52/503] COO
[53/503] CPRT
[54/503] GLW
[55/503] CPAY
[56/503] CTVA
[57/503] CSGP
[58/503] COST
[59/503] CTRA
[60/503] CRWD
[61/503] CCI
[62/503] CSX
[63/503] CMI
[64/503] CVS
[65/503] DHR
[66/503] DRI
[67/503] DVA
[68/503] DAY
[69/503] DECK
[70/503] DE
[71/503] DELL
[72/503] DAL

In [4]:
df_70_150.shape

(79, 133)

In [3]:
df_70_150.head()

,symbol,reportDate,date,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,revenue,...,cashflow_netCashUsedProvidedByFinancingActivities,cashflow_effectOfForexChangesOnCash,cashflow_netChangeInCash,cashflow_cashAtEndOfPeriod,cashflow_cashAtBeginningOfPeriod,cashflow_operatingCashFlow,cashflow_capitalExpenditure,cashflow_freeCashFlow,cashflow_link,cashflow_finalLink
0,BMY,2024-12-31,2024-12-31,USD,0000014272,2025-02-12,2025-02-12 12:52:08,2024,FY,48300000000,...,5127000000,-137000000,-1172000000,10347000000,11519000000,15190000000,-1248000000,13942000000,https://www.sec.gov/Archives/edgar/data/14272/...,https://www.sec.gov/Archives/edgar/data/14272/...
1,AVGO,2024-11-03,2024-11-03,USD,0001730168,2024-12-20,2024-12-20 17:26:46,2024,FY,51574000000,...,-1733000000,0,-4841000000,9348000000,14189000000,19962000000,-548000000,19414000000,https://www.sec.gov/Archives/edgar/data/173016...,https://www.sec.gov/Archives/edgar/data/173016...
2,BR,2024-06-30,2024-06-30,USD,0001383312,2024-08-06,2024-08-06 10:22:53,2024,FY,6506800000,...,-855500000,-600000,52100000,304400000,252300000,1056200000,-113000000,943200000,https://www.sec.gov/Archives/edgar/data/138331...,https://www.sec.gov/Archives/edgar/data/138331...
3,BRO,2024-12-31,2024-12-31,USD,0000079282,2025-02-13,2025-02-12 17:37:07,2024,FY,4705000000,...,-64000000,-13000000,199000000,2502000000,2303000000,1174000000,0,1174000000,https://www.sec.gov/Archives/edgar/data/79282/...,https://www.sec.gov/Archives/edgar/data/79282/...
4,BLDR,2024-12-31,2024-12-31,USD,0001316835,2025-02-20,2025-02-20 13:46:21,2024,FY,16400492000,...,-1074505000,0,87468000,153624000,66156000,1872692000,-380569000,1492123000,https://www.sec.gov/Archives/edgar/data/131683...,https://www.sec.gov/Archives/edgar/data/131683...


Ho runnato solo i primi 150, poi vanno mergiati tutti insieme

# -------- PROVA CON SOLO TICKER APPLE --------

In [13]:
import pandas as pd
import requests

# === CONFIG ===
API_KEY = "PJcSVU8gR4wG9S9HVANWyssG62n2M6Sy"
symbol = "AAPL"

# === FUNCTION TO FETCH DATA ===
def get_fmp_data(endpoint, symbol):
    url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{symbol}?limit=1&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]  # solo l'ultimo anno
    return {}

# === FETCH DATA ===
print(f"Fetching annual fundamentals for {symbol}")

income = get_fmp_data("income-statement", symbol)
balance = get_fmp_data("balance-sheet-statement", symbol)
cashflow = get_fmp_data("cash-flow-statement", symbol)

# === MERGE & CREATE DATAFRAME ===
merged = {
    "symbol": symbol,
    "reportDate": income.get("date", balance.get("date", cashflow.get("date", None))),
    **income,
    **{f"balance_{k}": v for k, v in balance.items()},
    **{f"cashflow_{k}": v for k, v in cashflow.items()}
}

df_apple = pd.DataFrame([merged])  # salva in DataFrame con una riga
print(df_apple.head())  # anteprima



Fetching annual fundamentals for AAPL
  symbol  reportDate        date reportedCurrency         cik fillingDate  \
0   AAPL  2024-09-28  2024-09-28              USD  0000320193  2024-11-01   

          acceptedDate calendarYear period       revenue  ...  \
0  2024-11-01 06:01:36         2024     FY  391035000000  ...   

   cashflow_netCashUsedProvidedByFinancingActivities  \
0                                      -121983000000   

   cashflow_effectOfForexChangesOnCash  cashflow_netChangeInCash  \
0                                    0                -794000000   

   cashflow_cashAtEndOfPeriod  cashflow_cashAtBeginningOfPeriod  \
0                 29943000000                       30737000000   

   cashflow_operatingCashFlow  cashflow_capitalExpenditure  \
0                118254000000                  -9447000000   

   cashflow_freeCashFlow                                      cashflow_link  \
0           108807000000  https://www.sec.gov/Archives/edgar/data/320193...   

       

In [14]:
df_apple.columns

Index(['symbol', 'reportDate', 'date', 'reportedCurrency', 'cik',
       'fillingDate', 'acceptedDate', 'calendarYear', 'period', 'revenue',
       ...
       'cashflow_netCashUsedProvidedByFinancingActivities',
       'cashflow_effectOfForexChangesOnCash', 'cashflow_netChangeInCash',
       'cashflow_cashAtEndOfPeriod', 'cashflow_cashAtBeginningOfPeriod',
       'cashflow_operatingCashFlow', 'cashflow_capitalExpenditure',
       'cashflow_freeCashFlow', 'cashflow_link', 'cashflow_finalLink'],
      dtype='object', length=133)

## ---------- VARIABILI IMPORTANTI ED ESEMPI ------------

### VARIABILI IMPORTANTI PER INCOME

Variabile | Descrizione | Frequenza reale di variazione

revenue | Ricavi totali della società | Trimestrale (earnings)

grossProfit | Ricavi - costo del venduto | Trimestrale

operatingIncome | Risultato operativo prima delle tasse e interessi | Trimestrale

netIncome | Utile netto | Trimestrale

eps (Earnings per Share) | Utile per azione | Trimestrale

interestExpense | Interessi passivi | Trimestrale

In [ ]:
# Esempio di output per income

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "revenue": 391035000000,
        "costOfRevenue": 210352000000,
        "grossProfit": 180683000000,
        "grossProfitRatio": 0.4620634982,
        "researchAndDevelopmentExpenses": 31370000000,
        "generalAndAdministrativeExpenses": 0,
        "sellingAndMarketingExpenses": 0,
        "sellingGeneralAndAdministrativeExpenses": 26097000000,
        "otherExpenses": 0,
        "operatingExpenses": 57467000000,
        "costAndExpenses": 267819000000,
        "interestIncome": 0,
        "interestExpense": 0,
        "depreciationAndAmortization": 11445000000,
        "ebitda": 134661000000,
        "ebitdaratio": 0.3443707085,
        "operatingIncome": 123216000000,
        "operatingIncomeRatio": 0.3151022287,
        "totalOtherIncomeExpensesNet": 269000000,
        "incomeBeforeTax": 123485000000,
        "incomeBeforeTaxRatio": 0.3157901467,
        "incomeTaxExpense": 29749000000,
        "netIncome": 93736000000,
        "netIncomeRatio": 0.2397125577,
        "eps": 6.11,
        "epsdiluted": 6.08,
        "weightedAverageShsOut": 15343783000,
        "weightedAverageShsOutDil": 15408095000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER BALANCE

Variabile | Descrizione | Frequenza reale

totalAssets | Totale attivo | Trimestrale

totalLiabilities | Totale passivo | Trimestrale

cashAndCashEquivalents | Cassa disponibile | Trimestrale

totalDebt | Totale debito (a breve e lungo termine) | Trimestrale

inventory | Scorte | Trimestrale

shareholderEquity | Capitale proprio | Trimestrale

In [ ]:
# Esempio di output per balance

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "cashAndCashEquivalents": 29943000000,
        "shortTermInvestments": 35228000000,
        "cashAndShortTermInvestments": 65171000000,
        "netReceivables": 66243000000,
        "inventory": 7286000000,
        "otherCurrentAssets": 14287000000,
        "totalCurrentAssets": 152987000000,
        "propertyPlantEquipmentNet": 45680000000,
        "goodwill": 0,
        "intangibleAssets": 0,
        "goodwillAndIntangibleAssets": 0,
        "longTermInvestments": 91479000000,
        "taxAssets": 19499000000,
        "otherNonCurrentAssets": 55335000000,
        "totalNonCurrentAssets": 211993000000,
        "otherAssets": 0,
        "totalAssets": 364980000000,
        "accountPayables": 68960000000,
        "shortTermDebt": 22511000000,
        "taxPayables": 26601000000,
        "deferredRevenue": 8249000000,
        "otherCurrentLiabilities": 50071000000,
        "totalCurrentLiabilities": 176392000000,
        "longTermDebt": 96548000000,
        "deferredRevenueNonCurrent": 0,
        "deferredTaxLiabilitiesNonCurrent": 0,
        "otherNonCurrentLiabilities": 35090000000,
        "totalNonCurrentLiabilities": 131638000000,
        "otherLiabilities": 0,
        "capitalLeaseObligations": 12430000000,
        "totalLiabilities": 308030000000,
        "preferredStock": 0,
        "commonStock": 83276000000,
        "retainedEarnings": -19154000000,
        "accumulatedOtherComprehensiveIncomeLoss": -7172000000,
        "othertotalStockholdersEquity": 0,
        "totalStockholdersEquity": 56950000000,
        "totalEquity": 56950000000,
        "totalLiabilitiesAndStockholdersEquity": 364980000000,
        "minorityInterest": 0,
        "totalLiabilitiesAndTotalEquity": 364980000000,
        "totalInvestments": 126707000000,
        "totalDebt": 106629000000,
        "netDebt": 76686000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER CASHFLOW

Variabile | Descrizione | Frequenza reale

operatingCashFlow | Flussi di cassa dalle operazioni | Trimestrale

capitalExpenditure | Investimenti in capitale (CAPEX) | Trimestrale

freeCashFlow | Cash flow operativo - CAPEX | Trimestrale

dividendsPaid | Dividendi distribuiti | Trimestrale

In [ ]:
# Esempio di output per cashflow
[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "netIncome": 93736000000,
        "depreciationAndAmortization": 11445000000,
        "deferredIncomeTax": 0,
        "stockBasedCompensation": 11688000000,
        "changeInWorkingCapital": 3651000000,
        "accountsReceivables": -5144000000,
        "inventory": -1046000000,
        "accountsPayables": 6020000000,
        "otherWorkingCapital": 3821000000,
        "otherNonCashItems": -2266000000,
        "netCashProvidedByOperatingActivities": 118254000000,
        "investmentsInPropertyPlantAndEquipment": -9447000000,
        "acquisitionsNet": 0,
        "purchasesOfInvestments": -48656000000,
        "salesMaturitiesOfInvestments": 62346000000,
        "otherInvestingActivites": -1308000000,
        "netCashUsedForInvestingActivites": 2935000000,
        "debtRepayment": -5998000000,
        "commonStockIssued": 0,
        "commonStockRepurchased": -94949000000,
        "dividendsPaid": -15234000000,
        "otherFinancingActivites": -5802000000,
        "netCashUsedProvidedByFinancingActivities": -121983000000,
        "effectOfForexChangesOnCash": 0,
        "netChangeInCash": -794000000,
        "cashAtEndOfPeriod": 29943000000,
        "cashAtBeginningOfPeriod": 30737000000,
        "operatingCashFlow": 118254000000,
        "capitalExpenditure": -9447000000,
        "freeCashFlow": 108807000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]